# Overlay of segmentation labels on original image

In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from improc.io import parse_collection, DCAccessor
DCAccessor.register()
from holoviews import opts

import param
import panel as pn
from inter_view.dashboards import SegmentationExportDashBoard

from skimage.io import imsave, imread
from skimage.segmentation import relabel_sequential

import holoviews as hv
hv.extension('bokeh', width=100)

# config
configure file parsing and channel display options. Remove the `intensity_bounds` for autoscaling

In [ ]:
basedir = '../../data/2D'
outdir = '../../data/2D_corrections' # output directory for files saved with gui
img_subdir = 'TIF_OVR_MIP'
segmentation_subdir = 'TIF_OVR_MIP_SEG'

data_pattern = '{platedir}/{layer}/{f1}_{f2}_{f3}_{row}{col:02d}_T{T:04d}F{field:03d}L{L:02d}A{A:02d}Z{zslice:02d}C{channel:02d}.{ext}'
index = ['layer', 'platedir','row', 'col', 'field', 'zslice', 'channel']

channel_config = {img_subdir:{'cmap':'gray','intensity_bounds':(200,18500), 'slider_limits':(0,2**16)},
                  segmentation_subdir:{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':0.5}}

opts.defaults(opts.Image('channel.{}'.format(segmentation_subdir), clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,256*256-1), tools=['hover']),
              opts.Image('channel', frame_width=1500))

# parse files
- multi-index levels will be available as drop-down menus in the gui

In [ ]:
df = parse_collection(os.path.join(basedir, data_pattern), index)

# filter groups that contains a segmentation image
df = df.groupby(['platedir','row', 'col', 'channel'],).filter(lambda x: segmentation_subdir in x.reset_index().layer.tolist())
df.reset_index(['field', 'zslice', 'channel'], inplace=True)
df = df.dc[[img_subdir, segmentation_subdir]]

df

# dashboard
To create a new training set by manually correcting errors:
1. Activate the "Box Edit Tool" and draw an ROI on the image (shift + click/drag)
1. Select "copy for training" and click "export" to copy the original image and its predicted segmentation. 
1. It will creates 3 sub-folders: raw, pred, annot. annot initially contains a copy of the prediction and needs to be manually corrected.


In [ ]:
def read_first_slice(path):
    '''Reads only the first slice
    
    (TIF_OVR contain a second tiles mask channel)
    '''    
    return imread(path, img_num=0)

def copy_for_training(path, imgs, cmaps, intensity_bounds, labels):
    print(path)
    basedir, filename = os.path.split(path)
    labels = [l.replace(segmentation_subdir, 'annot').replace(img_subdir, 'raw') for l in labels]
    print(labels)
    imgs = {key:val for key,val in zip(labels, imgs)}
    imgs['annot'] = relabel_sequential(imgs['annot'])[0]
    imgs['annot'] = imgs['annot'].astype(np.int16)
    imgs['pred'] = imgs['annot']
    
    for label, img in imgs.items():
#         outdir = os.path.join(basedir, label)
        os.makedirs(outdir, exist_ok=True)
        outpath = os.path.join(outdir, filename)            
        imsave(outpath, img, compress=5)
copy_for_training.name = 'copy for training'

db = SegmentationExportDashBoard(df=df,
                                 multi_select_levels=['layer'],
                                 loading_fun=read_first_slice,
                                 channel_config=channel_config,
                                 composite_channels=[img_subdir],
                                 overlay_channels=[segmentation_subdir],
                                 export_funs=[copy_for_training],
                                 out_folder=outdir)

db.panel()